# Persisting memory across Strands Agents sessions

In this example you will learn how to persist memory across different sessions in your Strands Agents. 

We will use the use case of an agent that does web search using a `duckduckgo` search API.

In this notebook, we will:
- Explore the capabilities of a memory-powered Strands agent.
- Learn how to store, retrieve, and list memories.
- Understand how to perform web searches via the agent.
- Interact with the agent in an interactive loop.


### Usage Examples

Storing memories:
```
Remember that I prefer tea over coffee
```

Retrieving memories:
```
What do I prefer to drink?
```

Listing all memories:
```
Show me everything you remember about me
```

### Tips for Memory Usage

- Be explicit when asking the agent to remember information
- Use specific queries to retrieve relevant memories
- Memory persistence enables more natural and contextual conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account and AWS credentials configured in the environment
* Anthropic Claude 3.7 enabled on Amazon Bedrock
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket and Amazon DynamoDB

Let's now install the requirement packages for our Strands Agent

In [ ]:
# Install the required packages
!uv pip install -r requirements.txt

In [ ]:

# Import Required Libraries
import os
import boto3
from strands import Agent, tool
from strands.models import bedrock
from strands_tools import mem0_memory

from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from sentence_transformers import SentenceTransformer

bedrock.DEFAULT_BEDROCK_MODEL_ID = "apac.anthropic.claude-3-7-sonnet-20250219-v1:0" #Optional: Set a default model for Bedrock
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # Initialize embedding model for vector operations

## Alternative: Direct OpenSearch Implementation

For users who prefer to use OpenSearch directly without the Mem0 abstraction layer, we'll also show how to create custom OpenSearch tools. This gives you more control over the vector storage and retrieval operations.


In [ ]:
# OR - Run the script to Create Opensearch Serverless resource in your AWS Account
#!sh prereqs/deploy_OSS.sh Mac Users
! "E:\Program Files\Git\bin\bash.exe" -c prereqs/deploy_OSS.sh 

In [ ]:
# You can manually define your Opensearch Host 
os.environ["OPENSEARCH_HOST"] = "please enter your open search host"

In [ ]:
# Option 1: Opensearch Serverless
from dotenv import load_dotenv
load_dotenv() # take Opensearch environment variables

In [ ]:
# Initialize OpenSearch client
def get_opensearch_client():
    """Initialize OpenSearch client with AWS authentication"""
    host = os.getenv('OPENSEARCH_HOST')
    region = os.getenv('AWS_REGION', 'ap-southeast-2')
    
    if not host:
        raise ValueError("OPENSEARCH_HOST environment variable not set")
    
    credentials = boto3.Session().get_credentials()
    auth = AWSV4SignerAuth(credentials, region, 'aoss')
    
    client = OpenSearch(
        hosts=[{'host': host, 'port': 443}],
        http_auth=auth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        pool_maxsize=20,
    )
    return client

# Create index if it doesn't exist
def ensure_index_exists(client, index_name="memory-index"):
    """Create the memory index if it doesn't exist"""
    if not client.indices.exists(index=index_name):  # Add index= keyword
        index_body = {
            "settings": {
                "index": {
                    "knn": True,
                    "knn.algo_param.ef_search": 100
                }
            },
            "mappings": {
                "properties": {
                    "content": {"type": "text"},
                    "user_id": {"type": "keyword"},
                    "timestamp": {"type": "date"},
                    "embedding": {
                        "type": "knn_vector",
                        "dimension": 384  # all-MiniLM-L6-v2 embedding dimension
                    }
                }
            }
        }
        client.indices.create(index=index_name, body=index_body)  # Add index= keyword
        print(f"Created index: {index_name}")
    else:
        print(f"Index {index_name} already exists")


In [ ]:
# Custom OpenSearch memory tools
@tool
def opensearch_memory_store(content: str, user_id: str, index_name: str = "memory-index") -> str:
    """Store information in OpenSearch for later retrieval.
    
    Args:
        content (str): The information to store
        user_id (str): Unique identifier for the user
        index_name (str): OpenSearch index name (default: memory-index)
        
    Returns:
        str: Success or error message
    """
    try:
        client = get_opensearch_client()
        ensure_index_exists(client, index_name)
        
        # Generate embedding for the content
        embedding = embedding_model.encode(content).tolist()
        
        # Create document
        doc = {
            "content": content,
            "user_id": user_id,
            #"timestamp": "now",
            "embedding": embedding
        }
        
        # Store document
        #doc_id = str(uuid.uuid4())
        response = client.index(
            index=index_name,
            #id=doc_id,
            body=doc
        )
        
        return f"Successfully stored memory with ID: {response}"
    except Exception as e:
        return f"Error storing memory: {str(e)}"

@tool 
def opensearch_memory_retrieve(query: str, user_id: str, index_name: str = "memory-index", top_k: int = 5) -> str:
    """Retrieve relevant memories from OpenSearch based on a query.
    
    Args:
        query (str): The query to search for
        user_id (str): Unique identifier for the user
        index_name (str): OpenSearch index name (default: memory-index)
        top_k (int): Number of top results to return (default: 5)
        
    Returns:
        str: Retrieved memories or error message
    """
    try:
        client = get_opensearch_client()
        
        # Generate embedding for the query
        query_embedding = embedding_model.encode(query).tolist()
        
        # Search for similar memories
        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "must": [
                        {"term": {"user_id": user_id}}
                    ],
                    "should": [
                        {
                            "knn": {
                                "embedding": {
                                    "vector": query_embedding,
                                    "k": top_k
                                }
                            }
                        }
                    ]
                }
            }
        }
        
        response = client.search(index=index_name, body=search_body)
        
        if response['hits']['total']['value'] == 0:
            return "No memories found for this query."
        
        memories = []
        for hit in response['hits']['hits']:
            score = hit['_score']
            content = hit['_source']['content']
            memories.append(f"Score: {score:.2f} - {content}")
        
        return "Retrieved memories:\\n" + "\\n".join(memories)
    except Exception as e:
        return f"Error retrieving memories: {str(e)}"

@tool
def opensearch_memory_list(user_id: str, index_name: str = "memory-index", limit: int = 10) -> str:
    """List all stored memories for a user.
    
    Args:
        user_id (str): Unique identifier for the user
        index_name (str): OpenSearch index name (default: memory-index)
        limit (int): Maximum number of memories to return (default: 10)
        
    Returns:
        str: List of all memories or error message
    """
    try:
        client = get_opensearch_client()
        
        search_body = {
            "size": limit,
            "query": {
                "term": {"user_id": user_id}
            },
        }
        
        response = client.search(index=index_name, body=search_body)
        
        if response['hits']['total']['value'] == 0:
            return "No memories found for this user."
        
        memories = []
        for hit in response['hits']['hits']:
            content = hit['_source']['content']
            #timestamp = hit['_source']['timestamp']
            memories.append(f"{content}")
        
        return f"All stored memories ({response['hits']['total']['value']} total):\\n" + "\\n".join(memories)
    except Exception as e:
        return f"Error listing memories: {str(e)}"


## Define System Prompt

The `SYSTEM_PROMPT` variable defines the behavior and capabilities of the memory agent. This prompt guides the agent to provide personalized responses based on stored memories and perform web searches when necessary.

In [ ]:
# Define a focused system prompt for memory operations
SYSTEM_PROMPT = """You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
"""

## Define Web Search Tool

The `websearch` tool using [Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search) function allows the agent to perform web searches. This function handles exceptions and returns search results or appropriate error messages.

## Alternative: Create Memory Agent with Direct OpenSearch

Here's an alternative implementation that uses direct OpenSearch operations instead of the Mem0 abstraction layer. This approach gives you more control over the vector storage and retrieval process.


In [ ]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## Create Memory Agent

We will now initialize the memory-focused agent using the defined tools and system prompt. The Strands agent is capable of:
1. Storing and retrieving memories based on context. It uses memory to create more personalized and contextual AI interactions.
2. Performing web searches using DuckDuckGo to give updated information.

In [ ]:
# Create an agent with direct OpenSearch memory tools
USER_ID = "PeterRabbit"
opensearch_memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[opensearch_memory_store, opensearch_memory_retrieve, opensearch_memory_list, websearch],
)


## Alternative: Demonstrate OpenSearch Memory Operations

The following examples demonstrate how to store, retrieve, and list memories using the direct OpenSearch implementation.

- **opensearch_memory_store**: Save important information directly to OpenSearch
- **opensearch_memory_retrieve**: Access relevant memories based on semantic similarity
- **opensearch_memory_list**: View all stored memories for a user

This approach provides more control over vector embeddings and search parameters.


In [ ]:
# Store initial memories using direct OpenSearch tools
print("Storing memories using OpenSearch...")

# Store user information
result1 = opensearch_memory_store(
    f"The user's name is {USER_ID}.", USER_ID)

print("Store result 1:", result1)

# Store user preferences  
result2 = opensearch_memory_store("I like to drink tea more than coffee.", USER_ID)
print("Store result 2:", result2)


In [ ]:
# Retrieve memories using semantic search
print("Retrieving memories using OpenSearch...")

# Retrieve information about user's name
retrieved_name = opensearch_memory_retrieve("What is the user's name?", USER_ID)
print("Retrieved name info:", retrieved_name)

# Retrieve information about user's preferences
retrieved_prefs = opensearch_memory_retrieve("What are the user's drink preferences?", USER_ID)
print("Retrieved preferences:", retrieved_prefs)


In [ ]:
# Ask the agent a question using OpenSearch memory
response = opensearch_memory_agent("What are the events happening in New York today?")
print("Agent response:", response)


In [ ]:
# List all stored memories using OpenSearch
print("Listing all stored memories...")
all_memories = opensearch_memory_list(USER_ID)
print("All memories:", all_memories)


## Interactive Agent Usage

Finally, we provide an interactive loop for users to interact with the memory agent. Users can store new memories, retrieve existing ones, or list all stored memories.

To test interactive usage: 
1. Install the requirements: `pip install -r requirements.txt`
1. Run the python file `personal_agent_with_memory.py`. 

## Conclusion

This notebook demonstrates how to create a personal agent with memory capabilities using the Strands framework. The agent can:

1. Store information about the user
2. Retrieve relevant memories based on context
3. Search the web for additional information
4. Provide personalized responses

By combining these capabilities, the agent can maintain context across conversations and provide more personalized assistance over time.

### Cleanup

Run this bash script to clean up the Opensearch Serverless resources. You don't need to run this if you used the "MEM0_PLATFORM_API".

In [ ]:
#!sh prereqs/cleanup_OSS.sh #Mac Users
! "E:\Program Files\Git\bin\bash.exe" -c prereqs/cleanup_OSS.sh 